------------------------------------------------
Choosing important features (feature importance)
--------------------------------------------------

Feature importance is the technique used to select features using a trained supervised classifier. When we train a classifier such as a decision tree, we evaluate each attribute to create splits; we can use this measure as a feature selector. Let’s understand it in detail.

Random forests are among the most popular machine learning methods thanks to their relatively good accuracy, robustness, and ease of use. They also provide two straightforward methods for feature selection—mean decrease impurity and mean decrease accuracy.

Otto Train data

You can download training dataset, train.csv.zip, from the https://www.kaggle.com/c/otto-group-product-classification-challenge/data and place the unzipped train.csv file in your working directory.

This dataset describes 93 obfuscated details of more than 61,000 products grouped into 10 product categories (for example, fashion, electronics, and so on). Input attributes are the counts of different events of some kind.

The goal is to make predictions for new products as an array of probabilities for each of the 10 categories, and models are evaluated using multiclass logarithmic loss (also called cross entropy).

In [1]:
from pandas import read_csv
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
np.random.seed(1)

In [2]:
#Function to create Train and Test set from the original dataset 
def getTrainTestData(dataset,split):
    np.random.seed(0) 
    training = [] 
    testing = []
    np.random.shuffle(dataset) 
    shape = np.shape(dataset)
    trainlength = np.uint16(np.floor(split*shape[0]))
    for i in range(trainlength): 
        training.append(dataset[i])
    for i in range(trainlength,shape[0]): 
        testing.append(dataset[i])
    training = np.array(training) 
    testing = np.array(testing)
    return training,testing

In [3]:
#Function to evaluate model performance
def getAccuracy(pre,ytest): 
    count = 0
    for i in range(len(ytest)):
        if ytest[i]==pre[i]: 
            count+=1
    acc = float(count)/len(ytest)
    return acc

In [5]:
#Load dataset as pandas data frame
data = read_csv('data/ottoTrain.csv')
#Extract attribute names from the data frame
feat = data.keys()
feat_labels = feat.get_values()
#Extract data values from the data frame
dataset = data.values
#Shuffle the dataset
np.random.shuffle(dataset)
#We will select 50000 instances to train the classifier
inst = 50000
#Extract 50000 instances from the dataset
dataset = dataset[0:inst,:]
#Create Training and Testing data for performance evaluation
train,test = getTrainTestData(dataset, 0.7)
#Split data into input and output variable with selected features
Xtrain = train[:,0:94] 
ytrain = train[:,94] 
shape = np.shape(Xtrain)
print("Shape of the dataset ",shape)
#Print the size of Data in MBs
print("Size of Data set before feature selection:",(Xtrain.nbytes/1e6),"MB")

Shape of the dataset  (35000, 94)
Size of Data set before feature selection: 26.32 MB


In [6]:
#Lets select the test data for model evaluation purpose
Xtest = test[:,0:94] 
ytest = test[:,94]
#Create a random forest classifier with the following Parameters
trees= 250
max_feat= 7
max_depth = 30
min_sample = 2
clf = RandomForestClassifier(n_estimators=trees, max_features=max_feat, max_depth=max_depth, 
min_samples_split= min_sample, random_state=0,n_jobs=-1)
#Train the classifier and calculate the training time
import time
start = time.time() 
clf.fit(Xtrain, ytrain) 
end = time.time()
#Lets Note down the model training time
print("Execution time for building the Tree is: %f"%(float(end)- float(start)))
pre = clf.predict(Xtest)
#Let's see how much time is required to train the model on the training dataset:
#Evaluate the model performance for the test data
acc = getAccuracy(pre, ytest)
print("Accuracy of model before feature selection is",(100*acc))

Execution time for building the Tree is: 11.645748
Accuracy of model before feature selection is 98.82
